In [1]:
import pandas as pd

In [44]:
df = pd.read_excel('ved/info-201.xlsx',sheet_name=[0,1])
df

{0:              Поле          Значение
 0        Предмет:       Информатика
 1         Группа:               201
 2  Преподаватель:  Сошников Дмитрий,
 1:                    ФИО  Оценка
 0        Иванов Сергей       3
 1      Петрищенко Пётр       2
 2       Грицаев Даниил       5
 3  Воропаев Константин       4
 4     Егоров Александр       3}

In [57]:
import psycopg

def conn():
    return psycopg.connect('dbname=stud user=postgres password=p@ssw0rd')

def get_prof(surname,name):
    with conn() as c:
        r = c.execute('SELECT ID FROM Person WHERE Surname=%s AND Name=%s',(surname,name)).fetchone()
        if r:
            return r[0]
        id = c.execute('SELECT MAX(ID)+1 FROM Person').fetchone()[0]
        if id is None:
            id = 1
        c.execute('INSERT INTO Person (Id, Surname, Name) VALUES (%s,%s,%s)',(id, surname,name))
        c.execute('INSERT INTO Professor (id) VALUES (%s)',(id,))
        return id

def ensure_group(x):
    with conn() as c:
        r = c.execute('SELECT ID FROM sgroup WHERE id=%s',(x,)).fetchone()
        if r:
            return
        c.execute('INSERT INTO sgroup (id, year) VALUES (%s,%s)',(x,x//100))

def get_stud(surname,name,group):
    with conn() as c:
        r = c.execute('SELECT ID FROM Person WHERE Surname=%s AND Name=%s',(surname,name)).fetchone()
        if r:
            return r[0]
        id = c.execute('SELECT MAX(ID)+1 FROM Person').fetchone()[0]
        if id is None:
            id = 1
        c.execute('INSERT INTO Person (Id, Surname, Name) VALUES (%s,%s,%s)',(id, surname,name))
        c.execute('INSERT INTO Student (id,sgroup) VALUES (%s,%s)',(id,group))
        return id
    
def get_subj(subj,lect_id):
    with conn() as c:
        r = c.execute('SELECT ID FROM Subject WHERE Title=%s',(subj,)).fetchone()
        if r:
            return r[0]
        id = c.execute('SELECT MAX(ID)+1 FROM Subject').fetchone()[0]
        if id is None:
            id = 1
        c.execute('INSERT INTO Subject (Id, Title, Lecturer_id) VALUES (%s,%s,%s)',(id, subj,lect_id))
        return id

In [58]:
def process_sheet(df):
    group = df[0].iloc[1,1]
    prof = df[0].iloc[2,1]
    ensure_group(group)
    pid = get_prof(*prof.split(' '))
    subj = df[0].iloc[0,1]
    subj_id = get_subj(subj,pid)
    with conn() as c:
        for i,x in df[1].iterrows():
            s = x['ФИО'].split(' ')
            sid = get_stud(s[0],s[1],group)
            c.execute('INSERT INTO Marks (Student_id, Subject_id, Professor_id, Mark) VALUES (%s,%s,%s,%s)',
                      (sid,subj_id,pid,x['Оценка']))


In [64]:
import os
for fn in os.listdir('ved'):
    print(f"Обработка {fn}")
    df = pd.read_excel(f'ved/{fn}',sheet_name=[0,1])
    process_sheet(df)

Обработка info-201.xlsx
ФИО       Иванов Сергей
Оценка                3
Name: 0, dtype: object
ФИО       Петрищенко Пётр
Оценка                  2
Name: 1, dtype: object
ФИО       Грицаев Даниил
Оценка                 5
Name: 2, dtype: object
ФИО       Воропаев Константин
Оценка                      4
Name: 3, dtype: object
ФИО       Егоров Александр
Оценка                   3
Name: 4, dtype: object
ФИО       Игумнова Александра
Оценка                      4
Name: 5, dtype: object
ФИО       Андреева Дарья
Оценка                 5
Name: 6, dtype: object
ФИО       Артемова Екатерина
Оценка                     3
Name: 7, dtype: object
Обработка info-202.xlsx
ФИО       Алексеева Антонина
Оценка                     4
Name: 0, dtype: object
ФИО       Сигнальный Петр
Оценка                  2
Name: 1, dtype: object
ФИО       Коловратов Даниил
Оценка                    2
Name: 2, dtype: object
ФИО       Киль Александр
Оценка                 4
Name: 3, dtype: object
ФИО       Матёрая Светлана
О

In [70]:
def query(x):
 with conn() as c:
        res = c.execute(x)   
        return pd.DataFrame(res, columns=[x.name for x in res.description])
    
query("""
select p.name, p.surname, m.mark, sj.title from marks m 
join subject sj on m.subject_id = sj.id 
join student st on m.student_id = st.id 
join person p on m.student_id = p.id
""")

,name,surname,mark,title
0,Сергей,Иванов,3,Информатика
1,Пётр,Петрищенко,2,Информатика
2,Даниил,Грицаев,5,Информатика
3,Константин,Воропаев,4,Информатика
4,Александр,Егоров,3,Информатика
5,Александра,Игумнова,4,Информатика
6,Дарья,Андреева,5,Информатика
7,Екатерина,Артемова,3,Информатика
8,Антонина,Алексеева,4,Информатика
9,Петр,Сигнальный,2,Информатика


In [76]:
query("""
select concat(p.name,' ',p.surname), st.sgroup, ph_marks.mark
from person p 
join student st on p.id = st.id
left outer join (select * from marks m where m.subject_id = 2) as ph_marks 
   on ph_marks.student_id = st.id
where st.sgroup = 202
""")

,concat,sgroup,mark
0,Антонина Алексеева,202,5.0
1,Петр Сигнальный,202,5.0
2,Даниил Коловратов,202,NaN
3,Александр Киль,202,NaN
4,Светлана Матёрая,202,NaN
5,Наталья Ермилова,202,4.0
6,Елена Москалёва,202,NaN
7,Антон Пигмальцев,202,NaN


In [77]:
query("""
select p.surname, 
  (select avg(m.mark) from marks m where m.student_id=st.id) as avg_mark  
from person p 
join student st on p.id = st.id
order by p.surname
""")

,surname,avg_mark
0,Алексеева,4.5000000000000000
1,Андреева,5.0000000000000000
2,Артемова,3.0000000000000000
3,Воропаев,4.0000000000000000
4,Грицаев,5.0000000000000000
5,Егоров,3.0000000000000000
6,Ермилова,4.5000000000000000
7,Иванов,3.0000000000000000
8,Игумнова,4.0000000000000000
9,Киль,4.0000000000000000


In [83]:
query("""
select * from st st cross join mrk mrk 
""")

,id,surname,id,subj,mark
0,1,Иванов,1,Физика,2
1,1,Иванов,1,Информат,4
2,1,Иванов,2,Физика,3
3,2,Петров,1,Физика,2
4,2,Петров,1,Информат,4
5,2,Петров,2,Физика,3
6,3,Сидоров,1,Физика,2
7,3,Сидоров,1,Информат,4
8,3,Сидоров,2,Физика,3
